<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex10/Hugo_Abonizio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Hugo Abonizio'
print(f'Meu nome é {nome}')

Meu nome é Hugo Abonizio


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.9 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 70.8 MB/s 
     |████████████████████████████████| 6.6 MB 65.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  8 19:05:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
import gc
from tqdm.auto import tqdm
from typing import List


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset:
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.examples = []

        for text in tqdm(texts):
          tokenized_text = tokenize(f'{tokenizer.cls_token} {text}', tokenizer)
          padding = [tokenizer.pad_token_id] * max(0, max_seq_length - len(tokenized_text) + 1)
          tokenized_text += padding

          for i in range(0, len(tokenized_text) - 1, max_seq_length):
            if i + max_seq_length < len(tokenized_text):
              self.examples.append(tokenized_text[i: i + max_seq_length + 1])
              continue

            self.examples.append(tokenized_text[-max_seq_length - 1:])

        self.examples = torch.LongTensor(self.examples)
        gc.collect()

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return example[:-1], example[1:]

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-08 19:06:45--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 142.250.128.128, 142.251.6.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   190MB/s    in 6.3s    

2022-06-08 19:06:52 (186 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [9]:
# Load datasets
max_seq_length = 9

train_examples = 500
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()[:100_000]

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
# print(f'Truncating to {max_lines} lines.')
# texts = texts[:max_lines]

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 100000 lines.


  0%|          | 0/99800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
import gc
gc.collect()

50

In [11]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 12445784
valid examples: 14245
test examples: 16058


In [20]:
import torch.nn.functional as F

class MultiHeadSelfAttentionLayer(nn.Module):
    def __init__(self, n_heads, dim, max_length):
        super().__init__()

        self.n_heads = n_heads
        self.dim = dim
        self.max_length = max_length

        self.W_q = nn.Linear(self.dim, self.dim, bias=False)
        self.W_k = nn.Linear(self.dim, self.dim, bias=False)
        self.W_v = nn.Linear(self.dim, self.dim, bias=False)
        self.W_o = nn.Linear(self.dim, self.dim, bias=False)
        self.feed_forward = nn.Sequential(
            nn.Linear(self.dim, self.dim),
            nn.ReLU(),
            nn.Linear(self.dim, self.dim),
        )
        self.layer_norm1 = nn.LayerNorm(self.dim, eps=1e-6)
        self.layer_norm2 = nn.LayerNorm(self.dim, eps=1e-6)

    def attention(self, q, k, v, mask=None):
        scores = torch.matmul(q, k.transpose(-1, -2))
        scores /= math.sqrt(self.dim // self.n_heads)

        if mask is not None:
          scores += mask[:, None, :]

        probs = F.softmax(scores, dim=-1)
        e = torch.matmul(probs, v)

        return e

    def forward(self, inputs, attention_mask):
        batch_size = inputs.size(0)
        residual = inputs

        q = self.W_q(inputs).reshape(batch_size, self.max_length, self.n_heads, self.dim // self.n_heads)
        k = self.W_k(inputs).reshape(batch_size, self.max_length, self.n_heads, self.dim // self.n_heads)
        v = self.W_v(inputs).reshape(batch_size, self.max_length, self.n_heads, self.dim // self.n_heads)

        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        output = self.attention(q, k, v, attention_mask)

        output = output.transpose(1, 2).contiguous()
        output = output.reshape(batch_size, self.max_length, self.dim)
        output = self.W_o(output)

        output = self.layer_norm1(residual + output)
        residual = output
        output = self.feed_forward(output)
        output = self.layer_norm2(residual + output)

        # output *= attention_mask[:, :, None]
        return output


class LanguageModel(nn.Module):
    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int, n_heads=1):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()

        self.pad_token_id = pad_token_id
        self.embeddings = nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
        self.positional_embeddings = nn.Embedding(max_seq_length, dim)

        self.layers = (
            [MultiHeadSelfAttentionLayer(n_heads=n_heads, dim=dim, max_length=max_seq_length).to(device)] +
            [MultiHeadSelfAttentionLayer(n_heads=n_heads, dim=dim, max_length=max_seq_length).to(device) for _ in range(n_layers - 1)]
        )

        self.lm_head = nn.Sequential(
            torch.nn.Linear(dim, dim),
            nn.ReLU(),
            torch.nn.Linear(dim, vocab_size, bias=False),
        )


    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """

        batch_size, max_seq_length = inputs.size()
        # seq_len = inputs.size(-1)
        pad_mask = (inputs == self.pad_token_id)
        # repeated_examples = inputs.unsqueeze(1).repeat(1, seq_len, 1)
        # attention_mask = torch.empty(repeated_examples.size(), device=inputs.device).fill_(float('-inf'))
        # repeated_examples = inputs.unsqueeze(1).repeat(1, seq_len, 1)
        attention_mask = torch.empty(batch_size, max_seq_length, max_seq_length, device=inputs.device).fill_(float('-inf'))
        attention_mask = attention_mask.triu(diagonal=1) # diagonal matrix
        attention_mask.masked_fill_(pad_mask.unsqueeze(1), float('-inf'))

        out = self.embeddings(inputs)
        pos_ids = torch.arange(inputs.size(1), dtype=torch.long, device=device)
        pos_embeddings = self.positional_embeddings(pos_ids)
        out += pos_embeddings

        for layer in self.layers:
          out = layer(out, attention_mask)

        return self.lm_head(out)

In [15]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=9,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

inputs = torch.tensor([[28,  2, 14,  5,  9, 23,  2, 14, 26], [10,  5,  9,  3, 10, 25, 0, 0, 0]], device=device)
model(inputs).shape

torch.Size([2, 9, 29794])

## Teste o modelo com um exemplo

In [21]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
    n_heads=4,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [17]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3818368


## Assert da Perplexidade


In [18]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              31156
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 200_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=256,
    n_layers=6,
    pad_token_id=tokenizer.pad_token_id,
    n_heads=4,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


best_ppl = float('inf')
train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

                if best_ppl > valid_ppl:
                  torch.save(model, 'best_model.pt')
                  best_ppl = valid_ppl

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 31378.25, valid ppl: 30394.70
10000 steps; 10240000 examples so far; train ppl: 358.12, valid ppl: 252.65
20000 steps; 20480000 examples so far; train ppl: 217.55, valid ppl: 213.37
30000 steps; 30720000 examples so far; train ppl: 191.50, valid ppl: 195.94
40000 steps; 40960000 examples so far; train ppl: 177.18, valid ppl: 185.15
50000 steps; 51200000 examples so far; train ppl: 167.59, valid ppl: 176.98


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 183.13469308757615


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz parte
Eu gosto de comer pizza pois me faz parte de
Eu gosto de comer pizza pois me faz parte de um
Eu gosto de comer pizza pois me faz parte de um dos
Eu gosto de comer pizza pois me faz parte de um dos anos
Eu gosto de comer pizza pois me faz parte de um dos anos,
Eu gosto de comer pizza pois me faz parte de um dos anos, o
Eu gosto de comer pizza pois me faz parte de um dos anos, o que
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o que
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o que o
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o que o que
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o que o que é
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o que o que é o
Eu gosto de comer pizza pois me faz parte de um dos anos, o que o que o que é o que
Eu gosto de comer pizza pois me faz parte de um 


## Teste com modelo maior

In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=256,
    n_layers=4,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


best_ppl = float('inf')
train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

                if best_ppl > valid_ppl:
                  torch.save(model, 'best_model_2.pt')
                  best_ppl = valid_ppl

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 31007.22, valid ppl: 29978.58
10000 steps; 10240000 examples so far; train ppl: 349.16, valid ppl: 247.75
20000 steps; 20480000 examples so far; train ppl: 211.98, valid ppl: 207.26


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.